In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'

In [2]:
# Importing the libraries:
import tensorflow as tf
from tensorflow import keras
import numpy as np
tf.__version__, keras.__version__, tf.config.list_physical_devices("GPU")

('2.10.0',
 '2.10.0',
 [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')])

# Prepping the Images:

In [3]:
# Creating a function to automate image loading:
batch_size = 32
def load_img(path = r"images/", batch = batch_size):
    data = tf.keras.preprocessing.image_dataset_from_directory(
        path,
        label_mode = None,
        color_mode = 'rgb',
        image_size = (100, 100),
        shuffle = True
    )
    data = data.unbatch()
    data = data.batch(batch, drop_remainder = True).shuffle(1000)
    for i in data:
        i = i/255.
    return data

In [4]:
data = load_img()

Found 63565 files belonging to 1 classes.


In [5]:
# Creating a image plotter:
def plot(x_train, y, image_no = 1):
    for i in tf.range(image_no):
        img = x_train[y]
        img = tf.keras.preprocessing.image.array_to_img(img)
        img.save(f"Saved Pictures/progress{y}.jpg")


# Creating a GAN:

In [6]:
# Sequential generator:
coding_size = 100
generator = keras.models.Sequential([
    keras.layers.Dense(25 * 25 * 128, input_shape = [coding_size]),
    keras.layers.Reshape([25, 25, 128]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(64, kernel_size = 5, strides = 2, padding = "same", activation = 'selu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(3, kernel_size = 5, strides = 2, padding = "same", activation = "tanh")
])

In [7]:
discriminator = tf.keras.models.Sequential([
    keras.layers.Conv2D(64, kernel_size = 5, strides = 2, padding = "same", activation = keras.layers.LeakyReLU(0.2), input_shape = [100, 100, 3]),
    keras.layers.Dropout(0.4),
    keras.layers.Conv2D(64, kernel_size = 5, strides = 2, padding = "same", activation = keras.layers.LeakyReLU(0.2)),
    keras.layers.Dropout(0.4),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation = "sigmoid")
])

In [8]:
# Creating a gan:
gan = keras.models.Sequential([generator, discriminator])

# Training the GAN:

In [9]:
# Creating some more variables:
optimizer = tf.keras.optimizers.Nadam()
loss_fn = tf.keras.losses.binary_crossentropy

In [10]:
# Compiling the model:
discriminator.compile(
    loss = loss_fn,
    optimizer = optimizer
)
discriminator.trainable = False
gan.compile(
    loss = loss_fn,
    optimizer = optimizer
)

In [11]:
#  Discriminator step:
def train_d(model, noise, x_batch, batch_size = batch_size):
    generator, discriminator = model.layers
    predictions = generator(noise)
    inputs = tf.concat([predictions, x_batch], axis = 0)
    y = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
    discriminator.trainable = True
    discriminator.fit(inputs, y)

In [12]:
# Generator step:
def train_g(model, noise, x_batch, batch_size = batch_size):
    discriminator = model.layers[1]
    discriminator.trainable = False
    y = tf.constant([[1.]] * batch_size)
    model.fit(noise, y)

In [13]:
# Predicting values:
def predict(model, epoch, batch_size = batch_size, coding_size = coding_size):
    noise = tf.random.normal([batch_size, coding_size])
    generator = model.layers[0]
    predictions = generator(noise)
    plot(predictions, epoch)

In [14]:
# Training Loop:
epochs = 100
for epoch in range(epochs):
    for x_batch in data:
        
        # Training the Discriminator:
        noise = tf.random.normal([batch_size, coding_size])
        train_d(gan, noise, x_batch)
        
        # Training the Generator:
        noise = tf.random.normal([batch_size, coding_size])
        train_g(gan, noise, x_batch)
    
    print(f"Epochs {epoch + 1} passed.")
    predict(gan, epoch + 1)

1/1 [==============================] - 0s 16ms/step - loss: 16.5228
